In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

import torchvision
import torchvision.models as models
from torchvision import *
from torchvision import datasets
from torchvision.models.feature_extraction import create_feature_extractor

from torchmetrics import *

from torch.utils.data import *

import sklearn as sk
from sklearn import *
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

from itertools import compress


import matplotlib.pyplot as plt


import yfinance as yf


In [ ]:
# time step to predict ahead, creates input sequences
# gonna have to figure out test
def create_lstm_data(data, time_step=1, future = 0,test_percent = .1):
    x_vec, y_vec = [], []
    # formats data so y = t and x = t-1, ... , t-time_steps
    for i in range(len(data) - time_step):
        split = i + time_step
        # checks if data is of same length for concatentation
        length = data[split : split + future, 0].shape[0]
        if(length == future):
            x_vec.append(data[i : split, 0].unsqueeze(0))
            y_vec.append(data[split : split + future, 0].unsqueeze(0))
    # calculate number of elements to allocate to test
    #dataset_length = len(x_vec)
    #num_of_train = dataset_length - (int)(test_percent * dataset_length)
    # concats x into matrix and y into vector, needs unsqueez to add single dimension for LSTM
    return torch.cat(x_vec,0).unsqueeze(-1), torch.cat(y_vec,0).unsqueeze(-1)#, torch.cat(x_vec[num_of_train:],0).unsqueeze(-1), torch.cat(y_vec[num_of_train:],0).unsqueeze(-1)

# create a dataset out of timeseries data, must be formatted first, tensor
# correct timeseries formatation
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,i):
        return self.X[i], self.y[i]
    

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout = 0.5):
        super(Encoder,self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.input_dim = input_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, n_layers, dropout=dropout,
                           batch_first=True, bidirectional = True)
        
        #to choose part of birdectinals were important
        self.fc_hidden = nn.Linear(hidden_dim*2, hidden_dim)
        self.fc_cell = nn.Linear(hidden_dim*2, hidden_dim)

    def forward(self, x):
        
        outputs, (hidden, cell) = self.rnn(x)
        #print(x.shape)
        print(hidden.view(-1,64,self.hidden_dim).shape)
        

        #chooses which way of directionsal is most important
        hidden = self.fc_hidden(torch.cat((hidden[0:1],hidden[1:2]), dim = 2))
        cell = self.fc_cell(torch.cat((cell[0:1],cell[1:2]), dim = 2))
        #print(hidden.shape)
        return hidden, cell

In [ ]:
# Download stock data from Yahoo Finance
hmc_data = yf.download('HMC', period='10y')['Close']
hymtf_data = yf.download('HYMTF', period='10y')['Close']
# get spread
spread = hmc_data - hymtf_data
spread = spread.values.reshape(-1,1)

In [ ]:
#parameters for data
BATCH_SIZE = 64
input_size = 30
output_size = 60

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
#transforms/normalizes data and converts it to a tensor
normalized_data = torch.tensor(scaler.fit_transform(spread))
X_train, y_train = create_lstm_data(normalized_data, input_size, output_size)
#creates dataset
train_dataset = TimeSeriesDataset(X_train,y_train)
#test_dataset = TimeSeriesDataset(X_test,y_test)
#puts dataset into dataloader
train_dataloader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE)
#test_dataloader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE)

In [ ]:
input_dim = 1
num_layers = 1
hidden_size = 16
dropout = 0.5
teacher_forcing = 0.2


device = device = 'cuda' if torch.cuda.is_available() else 'cpu'
encoder = Encoder(input_dim,num_layers,hidden_size,dropout)

In [ ]:
encoder()